In [2]:
# importing relevent libraries.
import requests
import re
import math
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import pandas as pd
import os


In [ ]:
links = []
cities = {'Karachi': 'Karachi-2-1', 'Islamabad': 'Islamabad-3-1',
          'Lahore': 'Lahore-1-1', 'Rawalpindi': 'Rawalpindi-41-1'}
city = 'Karachi'
for x in range(1, 100):
    link1 = "https://www.zameen.com/Flats_Apartments/"+cities[city]+".html"
    link2 = "https://www.zameen.com/Houses_Property/"+cities[city]+".html"
    link3 = "https://www.zameen.com/Rentals_Houses_Property/" + \
        cities[city]+".html"
    link4 = "https://www.zameen.com/Rentals_Flats_Apartments/" + \
        cities[city]+".html"
    link1 = link1[:-6] + str(x) + link1[-6+1:]
    links.append(link1)
    link2 = link2[:-6] + str(x) + link2[-6+1:]
    links.append(link2)
    link3 = link3[:-6] + str(x) + link3[-6+1:]
    links.append(link3)
    link4 = link4[:-6] + str(x) + link4[-6+1:]
    links.append(link4)


In [ ]:
pages = []
for x in range(len(links)):
    page = requests.get(links[x])
    soup = BeautifulSoup(page.content, 'html.parser')
    pages.append(soup)


In [ ]:
property_links = []
for i in range(len(pages)):
    page = pages[i].find_all("a", class_="_7ac32433")
    for x in range(len(page)):
        property_link = "https://www.zameen.com"+str(page[x]["href"])
        property_links.append(property_link)


In [ ]:
dict = {'links': property_links}
df = pd.DataFrame(dict)
df.to_csv('links.csv')  # saving links into CSV


In [ ]:
property_names, property_types, property_prices, property_purposes, property_locations, property_baths, property_beds = [
], [], [], [], [], [], []  # Declaring features variables as list.
property_cities, property_area_names, property_ids, property_descriptions, property_areas, property_dates_added = [], [], [], [], [], []
links_count = 2999
for x in property_links[links_count:]:
    if (links_count >= 5000):
        break
    page = requests.get(x)
    soup = BeautifulSoup(page.content, 'html.parser')
    print(x)

    if (soup.find(class_="_64bb5b3b") != None):
        property_name = soup.find(class_="_64bb5b3b").getText()
        property_type = soup.find(class_="_812aa185").getText()
        property_price = soup.find_all('span', attrs={'aria-label': 'Price'})
        property_area = soup.find_all('span', attrs={'aria-label': 'Area'})
        property_type = soup.find_all('span', attrs={'aria-label': 'Type'})
        property_purpose = soup.find_all(
            'span', attrs={'aria-label': 'Purpose'})
        property_location = soup.find_all(
            'span', attrs={'aria-label': 'Location'})
        property_bath = soup.find_all('span', attrs={'aria-label': 'Baths'})
        property_bed = soup.find_all('span', attrs={'aria-label': 'Beds'})
        property_date_added = soup.find_all(
            'span', attrs={'aria-label': 'Creation date'})

        # Standardization of price and area
        property_price = property_price[-1].getText()
        if ('Lakh' in property_price):
            property_price = property_price.replace('Lakh', '')
            property_price = property_price.replace('PKR', '')
            property_price = str(math.ceil(float(property_price))*100000)
        elif ('Crore' in property_price):
            property_price = property_price.replace('Crore', '')
            property_price = property_price.replace('PKR', '')
            property_price = str(math.ceil(float(property_price))*10000000)
        elif ('Thousand' in property_price):
            property_price = property_price.replace('Thousand', '')
            property_price = property_price.replace('PKR', '')
            property_price = str(math.ceil(float(property_price))*1000)
        property_type = property_type[-1].getText()
        property_purpose = property_purpose[-1].getText()
        property_location = property_location[-1].getText()
        property_bath = property_bath[-1].getText()
        property_bed = property_bed[-1].getText()
        property_description = soup.find(class_="_2a806e1e").getText()
        lst = property_location.split(',')
        property_area_name = lst[0]
        property_area = property_area[-1].getText()
        if ('Kanal' in property_area):
            property_area = property_area.replace('Kanal', '')
            property_area = str(float(property_area)*20)
        elif ('Marla' in property_area):
            property_area = property_area.replace('Marla', '')
        elif ('Sq. Yd.' in property_area):
            property_area = property_area.replace('Sq. Yd.', '')
            property_area = str(float(property_area)*0.03305785446)
        property_city = lst[-2]
        property_city = property_city.replace(' ', '')
        property_id = soup.find_all('span', attrs={'aria-label': 'Link name'})
        property_id = property_id[-1].getText()
        property_names.append(property_name)
        property_types.append(property_type)
        property_date_added = property_date_added[-1].getText()
        # Remove ago from the property_date_added
        property_date_added = property_date_added.replace(' ago', '')
        # If hours in property_date_added then convert to that date
        if 'hours' in property_date_added:
            property_date_added = datetime.now(
            ) - timedelta(hours=int(property_date_added.split(' ')[0]))
            property_date_added = property_date_added.strftime("%d/%m/%Y")
        # If days in property_date_added then convert to that date
        elif 'days' or 'day' in property_date_added:
            property_date_added = datetime.now(
            ) - timedelta(days=int(property_date_added.split(' ')[0]))
            property_date_added = property_date_added.strftime("%d/%m/%Y")
        # If weeks in property_date_added then convert to that date
        elif 'weeks' in property_date_added:
            property_date_added = datetime.now(
            ) - timedelta(weeks=int(property_date_added.split(' ')[0]))
            property_date_added = property_date_added.strftime("%d/%m/%Y")
        # If months in property_date_added then convert to that date
        elif 'months' in property_date_added:
            property_date_added = datetime.now(
            ) - timedelta(days=int(property_date_added.split(' ')[0]) * 30)
            property_date_added = property_date_added.strftime("%d/%m/%Y")
        # If years in property_date_added then convert to that date
        elif 'years' in property_date_added:
            property_date_added = datetime.now(
            ) - timedelta(days=int(property_date_added.split(' ')[0]) * 365)
            property_date_added = property_date_added.strftime("%d/%m/%Y")
        property_price = re.sub("[^0-9]", "", property_price)
        print(property_price)
        property_prices.append(int(property_price))
        property_purposes.append(property_purpose)
        property_locations.append(property_location)
        property_baths.append(property_bath)
        property_beds.append(property_bed)
        property_cities.append(property_city)
        property_area_names.append(property_area_name)
        property_ids.append(property_id)
        property_descriptions.append(property_description)
        property_areas.append(property_area)
        property_dates_added.append(property_date_added)
        links_count += 1


In [ ]:
# saving list in to dictionaries
dict = {'Property_ID': property_ids, 'Property_Name': property_names, 'Property_Type': property_types, 'Property_Purpose': property_purposes, 'Baths': property_baths, 'Bedrooms': property_beds, 'City': property_city,
        'Property_Location': property_locations, 'Area': property_areas, 'City': property_city, 'Area_Name': property_area_names, 'Properity_Price': property_prices, 'Date_Added': property_dates_added}

df = pd.DataFrame(dict)  # converting dictionary into Dataframe
i = 1
file_name = 'data_'+city+'_'+str(i)+'.csv'
while os.path.isfile(file_name):
    i += 1
    file_name = 'data_'+city+'_'+str(i)+'.csv'
df.to_csv(file_name)  # creating CSV from dataframe
